## Evaluación de los modelos para generar lenguaje

Para probar el entrenamiento de cada modelo  utilizamos el conjunto de prueba de la siguiente manera: cada oración que lo conforma es dividida a la mitad, se le pasa al modelo la primera mitad y es concatenada con la respuesta, por último se compara la oración original con la que acabamos de crear para ver que tanto cambiaron, claro que esto tiene un problema existen dialogos que únicamente tienen una palabra, en estos casos en realidad no tenemos con que comprar (cosa que se modifica para la red seq to seq)

Lo anterior aplica para el modelo de Bengio y RNN, para el último modelo el corpus de prueba es distinto, la idea es la misma pero aquí si tenemos la oración que siguiente

Inicialmente pensamos en utilizar la métrica de Levenshtein, pues esta nos diría qué tan distantes son las oraciones verificando los símbolos en ellas, pero esto puede calificar mal inicios de oraciones como "como estás" y "como te fue", aunque ambas son respuestas válidas, sin embargo nuestro corpus también contiene combinaciones de palabras específicas, como "si señor", "Torre Stark" y "Dr Strange", lo que sería calificado positivamente.

In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def get_bigrams(sentence):
    """
    Genera bigramas dada una cadena
    """
    words = sentence.split(' ')
    return list(zip(words,words[1:]))

def evaluate(s1, s2):
    """
    Evalua que tan diferentes son 2 cadenas
    Primero compara si alguna es sub cadena de otra,
    si eso no pasa evalua si existen digramas compartidos
    y hace un promedio para asignarle un valor entre 0 - 1
    donde 0 indica que las cadenas no s eparecen en nada 
    1 que son subcadenas
    """
    # Si una es subcadena de la otra, damos la calificacion mas alta: 1
    if s1 in s2 or s2 in s1:
        return 1
    # Ahora intentamos con pares
    bigrams = get_bigrams(s1)
    if len(bigrams) == 0:
        return 0
    num_bigrams_in = 0
    for bigram in bigrams:
        if ' '.join(bigram) in s2:
            num_bigrams_in += 1
    return num_bigrams_in / len(bigrams)

In [ ]:
evaluate('Tony stark, que tal tu dia?', 'Tony stark, que me cuentas?')

Cargamos los modelos y el conjunto de prueba

In [ ]:
test = pickle.load(open('./pickles/datasets/test.pkl','rb'))
predict_rnn = pickle.load(open('./pickles/predict/rnn.pkl','rb'))
predict_bengio = pickle.load(open('./pickles/predict/bengio.pkl','rb'))

In [ ]:
predict_rnn.insert(2, '')

In [ ]:
def plot_evals(test, predict):
    evals = []
    limit = len(test)
    newlimit = 0
    try:
        for i in range(limit - 1):
            evals.append(evaluate(test[i], predict[i]))
            newlimit = i
    except Exception:
        pass

    xpoints = np.arange(0, newlimit+1, 1, dtype=int)
    ypoints = np.array(evals)

    plt.plot(xpoints, ypoints)
    plt.show()

    return evals

In [ ]:
rnn_evals = plot_evals(test, predict_rnn)

In [ ]:
bengio_evals = plot_evals(test, predict_bengio)

In [ ]:
for i, score in enumerate(bengio_evals):
    if score != 1 and len(test[i].split(' ')) > 1:
        print(test[i])
        print(predict_bengio[i])
        break

Además recordemos que en la práctica 2 se hace el  análisis para la red de Bengio dando los siguientes resultados:

*Para el entrenamiento con el 10% de las palabras tenemos:*
```
Entropy: 9.29426577135724
Perplexity: 627.845498555216
```

*Para el entrenamiento con todo el corpus:*
```
Entropy: 12.874982418175305
Perplexity: 7512.005574123771
```


# Evaluacion del modelo Sequence to sequence

Nuestra red Sequence to Sequence si esta entrenada para que, dada una oración de entrada o petición, devuelva una oración de salida o respuesta.

Para evaluar estos modelos, vamos a calcular la entropía resultante con el set de prueba, mostrando la gráfica de como la entropia se ajusta con cada valor de prueba

In [ ]:
test_pairs = pickle.load(open('./pickles/seqtoseq/test_pairs.pkl','rb'))
test_pairs[:10]

In [ ]:
def get_entropy(test_data):
    '''
    Obtenemos la entropia promedio del modelo

    Args:
        test_data (list): conjunto de prueba
    '''
    H = 0.0
    Hs = []
    # calculamos entropia como el promedio de las probabilidades de cada oración
    for before, after in tqdm(test_data):
        inp = before_vectorization([before])
        _, state = encoder(inp)
        #Probabilidad de la cadena
        p_cad = 1
        for word in after.split(' '):
            dec_inp = after_vectorization([word])[:, :1]
            pred, state = decoder(dec_inp, state, training=False)
            pred = tf.nn.softmax(pred)
            p_cad *= pred[0][0][0].numpy()
        #Longitud de la cadena
        M = len(after.split(' '))
        #Obtenemos la entropía cruzada de la cadena
        if p_cad != 0:
            H -= (1./M)*(np.log(p_cad)/np.log(2))
        Hs.append(H)

    return H/len(test_data), Hs

entropy, history = get_entropy(test_pairs)
perplexity = 2**entropy
print(f'Entropy: {entropy}')
print(f'Perplexity: {perplexity}')

Los valores obtenidos fueron:
```
Entropy: 40.61154408156754
Perplexity: 1679936781077.174
```

Para graficar el histórico de la entropía, dividimos entre el número de oraciones procesadas en ese momento

In [ ]:
plt.plot(range(len(history)), list(map(lambda h, i: h/i, history, range(1,len(test_pairs)+1))))
plt.title('Entropy history')
plt.ylabel('entropy')
plt.xlabel('sentence index')
plt.savefig('entropy_seqtoseq.png')
plt.show()

<p align="center">
  <img src="img/entropy_seqtoseq.png" alt="Red Bengio"/>
</p>

# Conclusión

Mientras que las redes de Bengio y RNN fueron las primeras en implementarse, la realidad es que no estaban enfocadas al problema que queríamos resolver. La red Sequence to Sequence fue la que mejores resultados nos arrojó en términos de coherencia y semántica, puesto que estaba entrenada para responder a un cierto input.

Otras opciones que tenemos para mejorar el preprocesamiento del corpus omitiendo las palabras que no tengan valor real para las oraciones generadas (como "yeah"), o incluso cambiar el enfoque, como usar un Transformer.